## Query Translation - Decomposition
> 입력 질문을 작은 하위 질문으로 분해하는 방법

- Google의 IRC(Interleaved Retrieval with Chain of Thought reasoning)라는 연구는 검색과 사고 사슬(Chain of Thought) 추론을 결합한 방법
- 예로 들어, 'think machine learning'이라는 입력 질문을 세 개의 하위 문제로 분해했습니다: 'think machine', 'machine learning'. 그런 다음, 각각의 하위 문제를 개별적으로 해결하여 최종 솔루션을 도출

<div style="text-align: center;">
    <img src="./img/IRCoT.png" alt="Embedding" style="width: 25%; display: inline-block; margin: 0 2%;">
    <img src="./img/decomposition.png" alt="Indexing" style="width: 65%; display: inline-block; margin: 0 2%;">
</div>

### Environment

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Retriever

In [2]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


### Decomposition Prompt

In [3]:
from langchain.prompts import ChatPromptTemplate

# Decomposition Prompt
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOpenAI(temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the specific components that make up an LLM-powered autonomous agent system?',
 '3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous behavior?']

In [5]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [6]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [7]:
answer

'The main components of an LLM-powered autonomous agent system interact with each other in a coordinated manner to enable autonomous behavior. \n\n1. Planning: The agent utilizes task decomposition techniques like Chain of Thought (CoT) and Tree of Thoughts to break down complex tasks into smaller, manageable subgoals. This enables efficient handling of tasks and planning ahead.\n\n2. Subgoal Decomposition: By decomposing tasks into smaller steps, the agent can effectively manage and tackle complex tasks. This process sheds light on the model\'s thinking process and helps in transforming big tasks into multiple manageable tasks.\n\n3. Reflection and Refinement: The agent engages in self-criticism and self-reflection over past actions, learns from mistakes, and refines them for future steps. This iterative process of self-improvement enhances the quality of the final results.\n\n4. External Classical Planner (optional): In some cases, an external classical planner is used for long-horiz

- 아래 사진은 각각 2,3 번째 Deocomposition Query 에 대한 Input 이다.
- 최종적으로 Query 3 에 대한 Retrieval-augmentation 과 함께 Query 1과 2에 대한 응답이 input 에 들어가게 된다.

<div style="text-align: center;">
    <img src="./img/decomposition2.png" alt="Embedding" style="width: 45%; display: inline-block; margin: 0 2%;">
    <img src="./img/decomposition3.png" alt="Indexing" style="width: 45%; display: inline-block; margin: 0 2%;">
</div>